In [1]:
!pip install cloud-tpu-client==0.10 torch==1.12.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl
import torch
import torch_xla
import torch_xla.core.xla_model as xm
device = xm.xla_device()
print(device)

ERROR: torch_xla-1.12-cp37-cp37m-linux_x86_64.whl is not a supported wheel on this platform.

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


xla:0


In [2]:
import os
import random

class DataBuilder:
    def __init__(self, in_path, labels_72, labels_5):
        self.data = []
        for ldir in os.listdir(in_path):
            total = 0
            if "72" in ldir : limit = labels_72
            elif "5" in ldir : limit = labels_5
            for individuals in os.listdir(os.path.join(in_path, ldir)):
                if total<limit:
                    labels = os.listdir(os.path.join(in_path, ldir))
                    labels = list(set(labels)-{individuals})
                    random.shuffle(labels)
                    images = os.listdir(os.path.join(in_path, ldir, individuals))
                    anchor = "0.png"
                    images = list(set(images)-{anchor})
                    anchor = "/".join([in_path, ldir, individuals, anchor])
                    for i in range(len(images)):
                        pos = "/".join([in_path, ldir, individuals, images[i]])
                        if i != 0:
                            label = os.listdir(os.path.join(in_path, ldir, labels[i]))
                            random.shuffle(label)
                            label = label[0]
                            neg = "/".join([in_path, ldir, labels[i], label])
                        else:
                            label = os.listdir(os.path.join(in_path, ldir, labels[i]))[0]
                            neg = "/".join([in_path, ldir, labels[i], label])
                        self.data.append([anchor, pos, neg])
                    total += 1


path = "/kaggle/input/face-recognition"
# path = "/".join(path)
data = DataBuilder(path, 1000, 1000)

In [3]:
import random
data = data.data
random.shuffle(data)
for i in data[:5]:
    print(i)

['/kaggle/input/face-recognition/subjects_0-1999_72_imgs/1601/0.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/1601/69.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/1261/68.png']
['/kaggle/input/face-recognition/subjects_0-1999_72_imgs/1433/0.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/1433/59.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/584/3.png']
['/kaggle/input/face-recognition/subjects_0-1999_72_imgs/105/0.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/105/40.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/330/17.png']
['/kaggle/input/face-recognition/subjects_0-1999_72_imgs/1152/0.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/1152/58.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/1164/21.png']
['/kaggle/input/face-recognition/subjects_100000-133332_5_imgs/112573/0.png', '/kaggle/input/face-recognition/subjects_100000-133332_5_imgs/112573/3.png', '/kaggle/i

In [16]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode
from sklearn.model_selection import train_test_split

class FaceDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, index):
        anchor, positive, negative = self.data[index]
        anchor = read_image(anchor, mode=ImageReadMode.RGB)
        positive = read_image(positive, mode=ImageReadMode.RGB)
        negative = read_image(negative, mode=ImageReadMode.RGB)
        return torch.as_tensor(anchor, dtype=torch.float32), torch.as_tensor(positive, dtype=torch.float32), torch.as_tensor(negative, dtype=torch.float32)
    
    def __len__(self):
        return len(self.data)

In [ ]:
import torch
from torch import nn as nn
from torch import optim as optim
from torch.nn import functional as F
from torch.optim import lr_scheduler
class Model(nn.Module):
    def __init__(self, in_channels, embedding_size, batch_size):
        super(Model, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, in_channels, kernel_size=1, padding=0),
                                   nn.AvgPool2d(kernel_size=2, stride=2))
        

        self.b1 = nn.Conv2d(3, 64, kernel_size=1, padding="same")
        self.b2 = nn.Sequential(nn.Conv2d(in_channels, 92, kernel_size=1, padding="same"),
                                nn.Conv2d(92, 128, kernel_size=3, padding="same"))
        self.b3 = nn.Sequential(nn.Conv2d(in_channels, 24, kernel_size=1, padding="same"),
                                nn.Conv2d(24, 32, kernel_size=5, padding="same"))
        self.b4 = nn.Sequential(nn.MaxPool2d(kernel_size=3, padding=1, stride = 1),
                                nn.Conv2d(in_channels, 32, kernel_size=1, padding="same"))
        

        self.b5 = nn.Conv2d(256, 32, kernel_size=1, padding="same")
        self.b6 = nn.Sequential(nn.Conv2d(256, 128, kernel_size=1, padding="same"),
                                nn.Conv2d(128, 64, kernel_size=3, padding="same"))
        self.b7 = nn.Sequential(nn.Conv2d(256, 24, kernel_size=1, padding="same"),
                                nn.Conv2d(24, 16, kernel_size=5, padding="same"))
        self.b8 = nn.Sequential(nn.MaxPool2d(kernel_size=3, padding=1, stride = 1),
                                nn.Conv2d(256, 16, kernel_size=1, padding="same"))

        self.b5 = nn.Conv2d(256, 32, kernel_size=1, padding="same")
        self.b6 = nn.Sequential(nn.Conv2d(256, 128, kernel_size=1, padding="same"),
                                nn.Conv2d(128, 64, kernel_size=3, padding="same"))
        self.b7 = nn.Sequential(nn.Conv2d(256, 24, kernel_size=1, padding="same"),
                                nn.Conv2d(24, 16, kernel_size=5, padding="same"))
        self.b8 = nn.Sequential(nn.MaxPool2d(kernel_size=3, padding=1, stride = 1),
                                nn.Conv2d(256, 16, kernel_size=1, padding="same"))
        
        self.fc = nn.Sequential(nn.Linear(401408, 512, bias=True),
                                nn.Linear(512, embedding_size, bias=True))
        
        self.batch_size = batch_size
        
    def forward(self, x):
        x = self.conv(x)

        xb1 = self.b1(x)
        xb2 = self.b2(x)
        xb3 = self.b3(x)
        xb4 = self.b4(x)
        x = torch.cat([xb1, xb2, xb3, xb4], dim = 1)

        xb5 = self.b5(x)
        xb6 = self.b6(x)
        xb7 = self.b7(x)
        xb8 = self.b8(x)
        x = torch.cat([xb5, xb6, xb7, xb8], dim = 1)

        x = x.view(self.batch_size, -1)
        x = self.fc(x)

        return x

class TripletLoss:
    def __init__(self, margin, p):
        self.margin = margin
        self.p = p

    def loss(self, anchor, positive, negative):
        distance_ap = F.pairwise_distance(anchor, positive, p = self.p)
        distance_an = F.pairwise_distance(anchor, negative, p = self.p)
        tripletloss = torch.clamp(input=distance_ap+(self.margin)-distance_an, min=0.0).mean()
        return tripletloss

class Optimizer:
    def __init__(self, model, lr):
        self.model = model
        self.lr = lr
    
    def optimize(self):
        return optim.Adam(self.model.parameters(), lr=self.lr)

class Schedular:
    def __init__(self, optimizer, lr, step_size, gamma):
        self.optimizer = optimizer
        self.lr = lr
        self.step_size = step_size
        self.gamma = gamma
    
    def decay(self):
        return lr_scheduler.StepLR(self.optimizer, self.step_size, self.gamma)

In [ ]:
class Variables:
    def __init__(self, in_channels=3, embedding_size=256, batch_size=50, batch_size_test_val = 50, lr=0.001, margin=0.2, p=2, epochs=100, step_size=30, gamma=0.1):
        self.batch_size = batch_size
        self.lr = lr
        self.in_channels = in_channels
        self.emembedding_size =embedding_size
        self.p = p
        self.margin = margin
        self.epochs = epochs
        self.step_size = step_size
        self.gamma = gamma
        self.batch_size_test_val = batch_size_test_val


In [ ]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import torch
from torch import nn as nn
from torch import optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader


if __name__ == "__main__":
    # warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.init")


    train = data[0:70000]
    val = data[70000:72000]
    test = data[72000:75000]
    

    train = FaceDataset(train)
    val = FaceDataset(val)
    test = FaceDataset(test)
    variables = Variables()
    train = DataLoader(train, batch_size = variables.batch_size, shuffle=False)
    val = DataLoader(val, batch_size = variables.batch_size_test_val, shuffle=False)
    test = DataLoader(test, batch_size = variables.batch_size_test_val, shuffle=False)

    print(len(train))
    print(len(val))
    print(len(test))
    
    
    
    model = Model(in_channels=variables.in_channels, embedding_size=variables.emembedding_size, batch_size=variables.batch_size)
    tripletloss = TripletLoss(margin=variables.margin, p=variables.p)
    optimizer = Optimizer(model=model, lr=variables.lr).optimize()
    schedular = Schedular(optimizer, variables.lr, variables.step_size, variables.gamma).decay()
    device = "cuda"
    device = xm.xla_device()
    model.to(device)

    for epoch in range(0, variables.epochs):
        model.train()
        train_cost = 0.0
        loop = tqdm(enumerate(train), total=len(train), leave=True)
        for _, train_batch in loop:
            anc, pos, neg = train_batch
            anc, pos, neg = anc.to(device), pos.to(device), neg.to(device)
            anc = model.forward(anc)
            pos = model.forward(pos)
            neg = model.forward(neg)
            loss = tripletloss.loss(anc, pos, neg)
            train_cost += loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_description(f"[{epoch+1}/{variables.epochs}]")
            loop.set_postfix(loss=loss.item())
        avg_train_loss = train_cost/len(train)
        
        model.eval()
        with torch.no_grad():
            val_cost = 0.0
            loop_val = tqdm(enumerate(val), total=len(val), leave=True)
            for _, val_batch in loop_val:
                anc, pos, neg = val_batch
                anc, pos, neg = anc.to(device), pos.to(device), neg.to(device)
                anc = model.forward(anc)
                pos = model.forward(pos)
                neg = model.forward(neg)
                val_loss = tripletloss.loss(anc, pos, neg)
                val_cost += val_loss.item()
                loop_val.set_description(f"[{epoch+1}/{variables.epochs}]")
                loop_val.set_postfix(loss=val_loss.item())
            avg_val_loss = val_cost/len(val)
            print(f"Epoch {epoch+1}/{variables.epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")
        checkpoint_filename = f"model_epoch_{epoch+1}.pth"
        if avg_train_loss < 100:
            torch.save(model.state_dict(), checkpoint_filename)
        schedular.step()

In [20]:
import torch

torch.cuda.empty_cache()


In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
xm.rendezvous('clear_memory')  # This can help clear TPU memory
